In [ ]:
import junodch_utils_read_img as utils

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm.keras import TqdmCallback

import rasterio
import keras
from keras import layers

import umap
from umap.parametric_umap import ParametricUMAP

# Data preparation
### Fetch data from file

In [ ]:
folderName = "img/Sokoto/"
pathSatellite = folderName + "Sentinel-2.tif"
#pathSatellite = folderName + "Landsat-8.tif"
#pathNight = folderName + "Night VIIRS_1.tif"
pathNight = folderName + "lowres_night_1.tif"
pathValidation = folderName + "Population GHSL_1.tif"

aoi = utils.getImgBorder(pathSatellite)

# Fetch coords
dataCoords, dataRadiance = utils.getTilesCoordsPerimeter(pathNight, validThreshold=0, area=aoi)

print('Tiles:',dataCoords.shape[0])


In [ ]:
# Fetch images
lightMask = dataRadiance>25
idxDarkData = np.random.choice(np.arange(dataCoords.shape[0])[dataRadiance<=1], 1000, replace=False)

lightCoords = dataCoords[lightMask]
darkCoords = dataCoords[idxDarkData]
selectData = np.concatenate((lightCoords, darkCoords))

with rasterio.open(pathSatellite) as f:
  trainData, _ = utils.getEachImgFromCoord(f, selectData, True)
trainData = utils.formatData(trainData, res=64, toFloat=True)

print('Light Tile:',len(lightCoords))
print('dark Tile:',len(darkCoords))
print('Total train',trainData.shape)

train = trainData
validation = np.concatenate((dataRadiance[lightMask], dataRadiance[idxDarkData])).astype("float32") / 255

# UMAP

In [ ]:
trainDataUmap = trainData
autoencoder = keras.models.load_model('model/autoencoder_64x64_encoder_1024')
encoder = keras.Model(inputs=autoencoder.inputs, outputs=autoencoder.get_layer('encoder').output)
trainDataUmap = encoder.predict(trainDataUmap)
print(trainDataUmap.shape)

In [ ]:
input_shape=trainDataUmap.shape[1:]
print(input_shape)

encoderUMAP = keras.Sequential([
  #layers.Dense(units=64, activation='relu', input_shape=input_shape),
  layers.Dense(units=2, input_shape=input_shape),
])
decoderUMAP = keras.Sequential([
  #layers.Dense(64, activation='relu', input_shape=(2,)),
  layers.Dense(trainDataUmap.shape[1], activation='sigmoid', input_shape=(2,)),
])
lossFunction = keras.losses.MeanSquaredError() # l2

embedder = ParametricUMAP(
  encoder=encoderUMAP,
  #decoder=decoderUMAP,
  autoencoder_loss=True,
  #parametric_reconstruction_loss_fcn=lossFunction,
  dims=input_shape,
  parametric_reconstruction= True,
  parametric_embedding=True,
  #n_training_epochs = 1,
  loss_report_frequency=2,
  keras_fit_kwargs={
    "callbacks": [TqdmCallback(verbose=1)],
    "verbose": 0,
  },
  #reconstruction_validation=test,
  verbose=False,
)
t = 2000
#embedding = umap.UMAP(random_state=42).fit_transform(data_train[:t])
embedding = embedder.fit_transform(trainDataUmap)

fig, ax = plt.subplots()
ax.plot(embedder._history['loss'])
ax.set_ylabel('Cross Entropy')
ax.set_xlabel('Epoch')

In [ ]:
trainDataUmap = trainData
autoencoder = keras.models.load_model('model/autoencoder_64x64_encoder_1024')
encoder = keras.Model(inputs=autoencoder.inputs, outputs=autoencoder.get_layer('encoder').output)
trainDataUmap = encoder.predict(trainDataUmap)
print(trainDataUmap.shape)

umapTest = umap.UMAP()
embedding = umapTest.fit_transform(trainDataUmap)

fig, ax2 = plt.subplots(figsize=(10, 10))
sc = ax2.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c= [1]*len(dataCoords[lightMask]) + [0]*len(idxDarkData),
    cmap='rainbow',
    s=100,
    alpha=0.3,
    rasterized=True,
)
ax2.axis('equal')
ax2.set_title("UMAP in Tensorflow embedding", fontsize=20)
plt.colorbar(sc, ax=ax2)

In [ ]:
def displayAutoencoderUmapResults(autoencoder, dataInput, precision=0, isEmbedded=True):
  MAX_ON_ROW = 20
  total = dataInput.shape[0]
  nRow = (dataInput.shape[0] // MAX_ON_ROW) + 1
  nCol = MAX_ON_ROW if total > MAX_ON_ROW else total

  # Display original
  plt.figure(figsize=(30,nRow*2))
  for i in range(0, total):
    ax = plt.subplot(nRow, nCol, 1+i)
    plt.imshow(dataInput[i])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
  print("Original data:",dataInput.shape)
  plt.show()

  # Display reconstruction
  if isEmbedded:
    decoded_imgs = autoencoder.inverse_transform(autoencoder.transform(dataInput))
  else:
    test = []
    for d in dataInput:
      test.append(tf.reshape(d, [-1]))
    decoded_imgs = autoencoder.inverse_transform(autoencoder.transform(test))
  plt.figure(figsize=(30,nRow*2))
  print("Output data:",decoded_imgs.shape)
  for i in range(0, decoded_imgs.shape[0]):
    ax = plt.subplot(nRow, nCol, 1+i)
    if isEmbedded:
      decoded_img = decoded_imgs[i]
    else:
      decoded_img = tf.reshape(decoded_imgs[i], [64,64,3])
    plt.imshow(decoded_img)
    score = lossFunction(dataInput[i], decoded_img)
    plt.title(np.round(score,precision))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
  plt.show()

In [ ]:
displayAutoencoderUmapResults(embedder, trainData, precision=5, isEmbedded=False)

In [ ]:
with rasterio.open(pathSatellite) as s:
  validation, metaValid = utils.getEachImgFromCoord(s, test[0:1] + test[1104:1123]+ test[4000:4010]+ test[10000:10010], True)
  #validation, metaValid = utils.getEachImgFromCoord(s, test[0:1] + test[1104:1123]+ test[2944:2964]+ test[4000:4020]+ test[5000:5020]+ test[10000:10020], True)

displayAutoencoderUmapResults(embedder, utils.formatDataForAutoencoder(validation,res=64), precision=5, isEmbedded=False)

In [ ]:
embedder.save('model/autoencoder_64_GEN8_V2')

In [ ]:
# Display area test
with rasterio.open(pathSatellite) as s:
  validationTest, metaValidTest = utils.getEachImgFromCoord(s, test[0:10] + test[368:378] + test[736:746] + test[1104:1114] + test[1472:1482], True)

utils.displayTiles(validationTest, metaValidTest)